In [121]:
from Env import EnvLabyrinthe
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

In [122]:
env = EnvLabyrinthe(
    nblignes = 12,
    nbcolonnes = 12,
    alea = 1, # 0 : deterministe / 1 : stochastique
    proba_mur = 0.15,
    proba_blanc = 0.45,
    proba_vert = 0,
    proba_bleue = 0.2,
    proba_rouge = 0.2
)

env.render()

J 0 0 3 3 2 / 0 0 / 0 2 

0 3 3 2 / 0 0 3 2 / / / 

0 0 0 3 0 0 2 3 3 2 3 / 

3 0 2 0 0 0 0 0 0 2 0 0 

2 2 / 3 3 0 0 / 0 0 0 3 

3 3 2 3 3 / / 2 3 / 2 0 

2 / / / 3 2 2 3 0 2 3 2 

3 2 0 / 0 3 0 0 0 0 0 2 

0 2 / 0 / 0 3 3 2 2 / 2 

0 2 0 / 2 0 0 0 3 3 0 0 

3 0 0 0 0 2 2 0 0 0 / 0 

0 2 0 0 / 2 0 0 2 3 0 0 



In [123]:
MDPbi = env.getMDPBiObj()
print(MDPbi[2])
print('--')
for a in MDPbi[2] :
    print(a)
    print('--')
print(env.infos_state_dict[9])

[[(10, 0.6875, (-1, -1), False), (0, 0.0625, (-1, -1), False), (1, 0.0625, (-1, -1), False), (11, 0.0625, (0, -2), False), (18, 0.0625, (-1, -1), False), (19, 0.0625, (-1, -1), False)], [(19, 0.5, (-1, -1), False), (10, 0.0625, (-1, -1), False), (11, 0.0625, (0, -2), False), (12, 0.0625, (0, -2), False), (18, 0.0625, (-1, -1), False), (20, 0.0625, (-1, -1), False), (29, 0.0625, (0, -2), False), (30, 0.0625, (-1, -1), False), (31, 0.0625, (-2, 0), False)], [(21, 0.5625, (0, -2), False), (12, 0.0625, (0, -2), False), (13, 0.0625, (-2, 0), False), (20, 0.0625, (-1, -1), False), (22, 0.0625, (-1, -1), False), (31, 0.0625, (-2, 0), False), (32, 0.0625, (-1, -1), False), (33, 0.0625, (-1, -1), False)], [(2, 1.0, (0, 0), False)], [(2, 1.0, (0, 0), False)], [(2, 1.0, (0, 0), False)], [(2, 1.0, (0, 0), False)], [(2, 1.0, (0, 0), False)]]
--
[(10, 0.6875, (-1, -1), False), (0, 0.0625, (-1, -1), False), (1, 0.0625, (-1, -1), False), (11, 0.0625, (0, -2), False), (18, 0.0625, (-1, -1), False), (19

In [124]:
# len(MDPbi)
env.nb_states

122

In [125]:
discount_factor = 0.9

def get_pred_by_action(MDPbi,state,action) :
     
    list_pred_by_action = list()    
    
    for pred in range(len(MDPbi)) :
        
            
        for choice in MDPbi[pred][action] :

            if choice[0] == state  : # si le prochain état = state

                list_pred_by_action.append((choice,pred))
                
    return list_pred_by_action


def get_first_member_matrix_and_obj(MDPbi,discount_factor,nb_obs,nb_actions,infos_state_dict) :
    
    objective_matrix = np.zeros((2,nb_obs,nb_actions))
    # Autant de contraintes que d'état
    first_member_matrix = np.zeros((nb_obs,nb_obs,nb_actions)) # (state,pred,action)
    
    for state in range(nb_obs) :
        
        first_member_matrix[state,state,:] = 1
        
        # first_member_matrix[state,action] += 
        for action in range(len(MDPbi[state])) :
            
            if state == nb_obs-1 :
                objective_matrix[0,state,action] = 2 + 1e-8
                objective_matrix[1,state,action] = 2 + 1e-8
            elif infos_state_dict[state] == 2 : # bleu
                objective_matrix[0,state,action] = 2
                objective_matrix[1,state,action] = 0
            elif infos_state_dict[state] == 3 : # rouge
                objective_matrix[0,state,action] = 0
                objective_matrix[1,state,action] = 2
            else :
                objective_matrix[0,state,action] = 1
                objective_matrix[1,state,action] = 1
            
            pred_by_action = get_pred_by_action(MDPbi,state,action)
            
            for pred in pred_by_action : # pred : (predecesseur,proba,(r1,r2),done)

                first_member_matrix[state,pred[1],action] = first_member_matrix[state,pred[1],action] - (discount_factor * (pred[0][1]))
    
    return first_member_matrix,objective_matrix

infos_state_dict = env.infos_state_dict
nb_states = env.nb_states
nb_actions = 8

"""
mdpbi_exemple = {
    0 : [ [(0,0.75,(0,0),False) , (1,0.25,(0,0),False)] , [ (0,0.5,(0,0),False) , (1,0.5,(0,0),False) ] , [(2,1.0,(0,0),False)]],
    1 : [ [(0,0.5,(0,0),False) , (1,0.5,(0,0),False)] , [ (0,0.25,(0,0),False) , (1,0.75,(0,0),False) ] , [(2,1.0,(0,0),False)]],
    2 : [ [(1,0.5,(0,0),False) , (2,0.5,(0,0),False)] , [ (2,0.25,(0,0),False) , (1,0.75,(0,0),False) ] , [(2,1.0,(0,0),False)]]
}

infos_state_dict = { 0 : 0 , 1 : 1, 2 : 2}
nb_states = 3
nb_actions = 3

print(nb_states,nb_actions)

MDPbi = mdpbi_exemple
"""
first_member_matrix,objective_matrix = get_first_member_matrix_and_obj(MDPbi,discount_factor,nb_states,nb_actions,infos_state_dict)

In [126]:
"""
objective_matrix = np.array([
    [[8,12,4],[11,9,2], [1e3,1e3,1e3]],
    [[13,6,5],[7,15,1], [1e3,1e3,1e3]]
])
"""
print(objective_matrix)
print("--")
print(first_member_matrix[0])
# print(first_member_matrix[23])
print("--")
#print(first_member_matrix.reshape((env.nb_states,env.nb_states * 8)))

[[[1.         1.         1.         ... 1.         1.         1.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [2.00000001 2.00000001 2.00000001 ... 2.00000001 2.00000001 2.00000001]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  ...
  [2.         2.         2.         ... 2.         2.         2.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [2.00000001 2.00000001 2.00000001 ... 2.00000001 2.00000001 2.00000001]]]
--
[[ 0.1      0.1      1.       1.       0.1      0.1      0.1      0.1    ]
 [ 0.       0.    

In [127]:
print(MDPbi[1])

[[(1, 1.0, (0, 0), False)], [(18, 0.6875, (-1, -1), False), (10, 0.0625, (-1, -1), False), (11, 0.0625, (0, -2), False), (19, 0.0625, (-1, -1), False), (29, 0.0625, (0, -2), False), (30, 0.0625, (-1, -1), False)], [(20, 0.5, (-1, -1), False), (11, 0.0625, (0, -2), False), (12, 0.0625, (0, -2), False), (13, 0.0625, (-2, 0), False), (19, 0.0625, (-1, -1), False), (21, 0.0625, (0, -2), False), (30, 0.0625, (-1, -1), False), (31, 0.0625, (-2, 0), False), (32, 0.0625, (-1, -1), False)], [(13, 0.5625, (-2, 0), False), (2, 0.0625, (-1, -1), False), (3, 0.0625, (0, -2), False), (4, 0.0625, (0, -2), False), (12, 0.0625, (0, -2), False), (20, 0.0625, (-1, -1), False), (21, 0.0625, (0, -2), False), (22, 0.0625, (-1, -1), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)], [(1, 1.0, (0, 0), False)]]


In [128]:
a = first_member_matrix # .reshape((nb_states,nb_states * nb_actions))
b = [1.0/(nb_states-1) for _ in range(nb_states)]
b[(nb_states-1)] = 0.0
# b[0] = 1.0
c = objective_matrix # .reshape((2,nb_states*nb_actions))

print("a shape : ",a.shape)
print("b shape : ",np.array(b).shape)
print("c shape : ",c.shape)

nbcont = nb_states
nbvar = nb_states * nb_actions

lignes = range(nbcont)
colonnes = range(nbvar)

print("nb_cont : ",nbcont)
print("nb_colonnes : ",nbvar)

a shape :  (122, 122, 8)
b shape :  (122,)
c shape :  (2, 122, 8)
nb_cont :  122
nb_colonnes :  976


In [113]:
"""
nbcont=2 
nbvar=4

# Range of plants and warehouses
lignes = range(nbcont)
colonnes = range(nbvar)

# Matrice des contraintes
a = np.array([[5.0/8.0,-1.0/8.0],
[3.0/4.0, -1.0/4.0],
[-1.0/4.0, 3.0/4.0],
[-1.0/8.0, 5.0/8.0]]).T

# Second membre
b = [1.0, 0.0]

# Coefficients de la fonction objectif
c = [[8.0, 12.0, 11.0,9.0],
 [13.0, 6.0, 7.0,15.0]]
"""

'\nnbcont=2 \nnbvar=4\n\n# Range of plants and warehouses\nlignes = range(nbcont)\ncolonnes = range(nbvar)\n\n# Matrice des contraintes\na = np.array([[5.0/8.0,-1.0/8.0],\n[3.0/4.0, -1.0/4.0],\n[-1.0/4.0, 3.0/4.0],\n[-1.0/8.0, 5.0/8.0]]).T\n\n# Second membre\nb = [1.0, 0.0]\n\n# Coefficients de la fonction objectif\nc = [[8.0, 12.0, 11.0,9.0],\n [13.0, 6.0, 7.0,15.0]]\n'

In [114]:
from gurobipy import *

m = Model("mdp")     
        
# declaration variables de decision
x = []
for i in range(nb_states):
    l = list()
    for j in range(nb_actions) :
        l.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name=(f"x{i+1}_{j+1}") ))
    x.append(l)
z = m.addVar(vtype=GRB.CONTINUOUS,lb=0 ,name="z")

# maj du modele pour integrer les nouvelles variables
m.update()

obj = LinExpr();
obj = z

In [115]:
print(c[0])

[[1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [2.         2.         2.         2.         2.         2.
  2.         2.        ]
 [1.         1.         1.         1.         1.         1.
  1. 

In [116]:
# definition de l'objectif
m.setObjective(obj,GRB.MAXIMIZE)

l= [1.0 for _ in range(2)]

m.addConstr(quicksum(c[0][i][j]*x[i][j] for j in range(nb_actions) for i in range(nb_states)) - z >= 0, "Controbj 1" )
m.addConstr(quicksum(c[1][i][j]*x[i][j] for j in range(nb_actions) for i in range(nb_states)) - z >= 0, "Controbj 2" )

# Definition des contraintes
for i in lignes:
    m.addConstr(quicksum(a[i][s][j]*x[s][j] for j in range(nb_actions) for s in range(nb_states)) == b[i], "Contrainte %d" % i)
        #print(quicksum(a[i][j]*x[j] for j in colonnes) ,"==",b[i])
        #print("----------------------------------------")

m.update()

In [117]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 87 rows, 681 columns and 4607 nonzeros
Model fingerprint: 0x374299ab
Coefficient statistics:
  Matrix range     [6e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e-02]
Presolve removed 0 rows and 195 columns
Presolve time: 0.02s
Presolved: 87 rows, 486 columns, 4090 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+30   2.000000e+30   4.000000e+00      0s
     179    1.5178954e+01   0.000000e+00   0.000000e+00      0s

Solved in 179 iterations and 0.06 seconds (0.01 work units)
Optimal objective  1.517895393e+01


In [118]:
for i in range(nb_states):
    for j in range(nb_actions) :
        print(x[i][j].x, end=" ")
    print(" ")

0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.0 0.0 0.0 0.1462100074404762 0.0  
0.0 0.0 0.014621000744047617 0.0 0.0 0.0 0.0 0.0  
0.01257440476190476 0.0 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.01257440476190476 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.015452538631346577 0.0 0.0 0.0 0.0  
0.01478289577420372 0.0 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.01257440476190476 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.023902082188924156 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.03495537459963844 0.0 0.0 0.0 0.0 0.0  
0.022139860607328872 0.0 0.0 0.0 0.0 0

In [119]:
d = np.array(x).reshape((nb_states,nb_actions))


for i in range(nb_states) :
    for j in range(nb_actions) :
        d[i,j] = d[i,j].x

print(d)

print(d.sum(axis = 1))   

for i in range(nb_states) :
    d[i,:] /= d.sum(axis = 1)[i]
        
        
print("----")
print(d)

[[0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.011904761904761904 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 0.1462100074404762 0.0]
 [0.0 0.0 0.014621000744047617 0.0 0.0 0.0 0.0 0.0]
 [0.01257440476190476 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.01257440476190476 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.0 0.015452538631346577 0.0 0.0 0.0 0.0]
 [0.01478289577420372 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.01257440476190476 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.023902082188924156 0.0 0.0 0.0 0.0 0.0]
 [0.0 0.0 0.03495537459963844 0.0 0.0 0.0 0.0 0.0]
 [0.022139860607328

In [81]:
def get_policy_from_pl(matrix) :
    return np.array(matrix).argmax(axis=1)

policy = get_policy_from_pl(d)

In [82]:

def proportion_rouge_bleu(env,policy) :
    
    nb_bleue = 0
    nb_rouge = 0
    
    state = env.replace_player_init()
    done = False
    cum_sum = 0
    i = 0
    max_iter = 30
    while not done and i < max_iter: 
        
        new_state,reward,done = env.step(np.random.choice(a=np.arange(8),p=np.array(policy[state,:], dtype=np.float32)))
        
        if env.infos_state_dict[new_state] == 2:
            nb_bleue += 1
        elif env.infos_state_dict[new_state] == 3:
            nb_rouge += 1
        
        cum_sum += reward
        state = new_state
        i += 1
    return nb_bleue,nb_rouge,i < max_iter



l_b = 0
l_r = 0

from tqdm import tqdm

for i in tqdm(range(1000)) : 

    nb_bleue,nb_rouge,_ = proportion_rouge_bleu(env,d)
    l_b += nb_bleue
    l_r += nb_rouge


nb_r = 0
nb_b = 0
for state in MDPbi :
    
    if env.infos_state_dict[state] == 2 :
        nb_b += 1
    elif env.infos_state_dict[state] == 3 :
        nb_r += 1

print(l_b,l_r)
print(f'{l_b/(l_b+l_r)}% de bleue et {l_r/(l_b+l_r)}% de rouge pour la simulation')
print(f'{nb_b/(nb_b+nb_r)}% de bleue et {nb_r/(nb_b+nb_r)}% de rouge dans le MDP')
print(nb_b,nb_r)

100%|█████████████████████████████████████| 1000/1000 [00:00<00:00, 1731.12it/s]

1719 1445
0.543299620733249% de bleue et 0.45670037926675094% de rouge pour la simulation
0.46808510638297873% de bleue et 0.5319148936170213% de rouge dans le MDP
22 25


In [83]:
state = env.replace_player_init()
env.render()
done = False
cum_sum = 0
i = 0
max_iter = 30
while not done and i < max_iter: 
    new_state,reward,done = env.step(int(policy[state]))
    cum_sum += reward
    print("------")
    env.render()
    state = new_state
    i += 1
    
print("-------------")
env.render()
print(f"test reward : {cum_sum}")

J 0 0 2 3 0 3 3 2 / 

3 3 0 0 0 0 3 0 3 2 

0 / 0 0 0 0 2 0 3 2 

3 0 0 0 / 0 3 3 3 0 

2 / 0 / 0 2 2 0 2 0 

2 2 3 2 0 0 3 3 0 0 

0 0 0 / 2 0 2 2 / 2 

0 0 0 2 0 3 3 0 0 3 

2 2 2 3 3 3 3 0 2 0 

3 0 0 0 3 2 3 0 0 0 

------
0 0 0 2 3 0 3 3 2 / 

3 3 J 0 0 0 3 0 3 2 

0 / 0 0 0 0 2 0 3 2 

3 0 0 0 / 0 3 3 3 0 

2 / 0 / 0 2 2 0 2 0 

2 2 3 2 0 0 3 3 0 0 

0 0 0 / 2 0 2 2 / 2 

0 0 0 2 0 3 3 0 0 3 

2 2 2 3 3 3 3 0 2 0 

3 0 0 0 3 2 3 0 0 0 

------
0 0 0 2 3 0 3 3 2 / 

3 3 0 0 0 0 3 0 3 2 

0 / 0 0 0 0 2 0 3 2 

3 0 0 J / 0 3 3 3 0 

2 / 0 / 0 2 2 0 2 0 

2 2 3 2 0 0 3 3 0 0 

0 0 0 / 2 0 2 2 / 2 

0 0 0 2 0 3 3 0 0 3 

2 2 2 3 3 3 3 0 2 0 

3 0 0 0 3 2 3 0 0 0 

------
0 0 0 2 3 0 3 3 2 / 

3 3 0 0 0 0 3 0 3 2 

0 / 0 0 0 0 2 0 3 2 

3 0 0 0 / 0 3 3 3 0 

2 / 0 / 0 J 2 0 2 0 

2 2 3 2 0 0 3 3 0 0 

0 0 0 / 2 0 2 2 / 2 

0 0 0 2 0 3 3 0 0 3 

2 2 2 3 3 3 3 0 2 0 

3 0 0 0 3 2 3 0 0 0 

------
0 0 0 2 3 0 3 3 2 / 

3 3 0 0 0 0 3 0 3 2 

0 / 0 0 0 0 2 0 3 2 

3 0 0 0 / 0 3 3 3 0 

2 / 

## Politique pures

In [138]:
from gurobipy import *

m = Model("mdp")     
        
# declaration variables de decision
x = []
for i in range(nb_states):
    l = list()
    for j in range(nb_actions) :
        l.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name=(f"x{i+1}_{j+1}") ))
    x.append(l)

d = []
for i in range(nb_states):
    l = list()
    for j in range(nb_actions) :
        l.append(m.addVar(vtype=GRB.BINARY, name=(f"d{i+1}_{j+1}") ))
    d.append(l)    

z = m.addVar(vtype=GRB.CONTINUOUS ,name="z")

# maj du modele pour integrer les nouvelles variables
m.update()

obj = LinExpr();
obj = z

In [139]:
# definition de l'objectif
m.setObjective(obj,GRB.MAXIMIZE)

m.addConstr(quicksum(c[0][i][j]*x[i][j] for j in range(nb_actions) for i in range(nb_states))  >= z, "Controbj 1" )
m.addConstr(quicksum(c[1][i][j]*x[i][j] for j in range(nb_actions) for i in range(nb_states))  >= z, "Controbj 2" )

# Definition des contraintes
for i in lignes:
    m.addConstr(quicksum(a[i][s][j]*x[s][j] for j in range(nb_actions) for s in range(nb_states)) == b[i], "Contrainte %d" % i)

for i in range(nb_states) :
    for j in range(nb_actions) :
        m.addConstr((1 - discount_factor) * x[i][j] - d[i][j] <= 0, "cm %d" % i)

for i in range(nb_states) :   
    m.addConstr(quicksum(d[i][j] for j in range(nb_actions)) <= 1.0 , "dd %d" % i)    

    
print(f"nb contraintes : {nb_states + (nb_states*nb_actions) + nb_states + 2}")    
print(f"nb_variables : {1 + (nb_states*nb_actions)*2}")    

m.update()
print(m)

nb contraintes : 1222
nb_variables : 1953
<gurobi.Model MIP instance mdp: 1222 constrs, 1953 vars, No parameter changes>


In [135]:
# autre maniere


        
m = Model("mdp")     

# declaration variables de decision
x = []
for i in range(nb_states):
    l = list()
    for j in range(nb_actions) :
        l.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name=(f"x{i+1}_{j+1}") ))
    x.append(l)

d = []
for i in range(nb_states):
    l = list()
    for j in range(nb_actions) :
        l.append(m.addVar(vtype=GRB.BINARY, lb=0, name=(f"d{i+1}_{j+1}") ))
    d.append(l)    

z = m.addVar(vtype=GRB.CONTINUOUS,lb=0 ,name="z")



m.update()



obj = LinExpr();
obj = 0
for reward in [0,1]:
    for j in range(nb_actions) :
        for i in range(nb_states):
            obj += c[reward][i][j]*x[i][j]


# Definition des contraintes
for i in lignes:
    m.addConstr(quicksum(a[i][s][j]*x[s][j] for j in range(nb_actions) for s in range(nb_states)) == b[i], "Contrainte %d" % i)

for i in range(nb_states) :
    for j in range(nb_actions) :
        m.addConstr((1 - discount_factor) * x[i][j] <= d[i][j], "cm %d" % i)

for i in range(nb_states) :   
    m.addConstr(quicksum(d[i][j] for j in range(nb_actions)) <= 1.0 , "dd %d" % i)    


print(f"nb contraintes : {nb_states + (nb_states*nb_actions) + nb_states + 2}")    
print(f"nb_variables : {1 + (nb_states*nb_actions)*2}")    

m.update()

nb contraintes : 1222
nb_variables : 1953


In [140]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1222 rows, 1953 columns and 9945 nonzeros
Model fingerprint: 0x09e2348f
Variable types: 977 continuous, 976 integer (976 binary)
Coefficient statistics:
  Matrix range     [6e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-03, 1e+00]
Presolve removed 122 rows and 122 columns
Presolve time: 0.01s
Presolved: 1100 rows, 1831 columns, 9697 nonzeros
Variable types: 977 continuous, 854 integer (854 binary)

Root relaxation: objective 1.530912e+01, 1392 iterations, 0.08 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   15.30912    0    2          -   15.30912      -     -    0s
H    0     0                      15.2777279   15.30912  0.21%     - 

In [142]:
for i in range(nb_states):
    for j in range(nb_actions) :
        print(x[i][j].x, end=" ")
    print(" ")


d = np.array(x).reshape((nb_states,nb_actions))


for i in range(nb_states) :
    for j in range(nb_actions) :
        d[i,j] = d[i,j].x

print(d)

print(d.sum(axis = 1))   

for i in range(nb_states) :
    d[i,:] /= d.sum(axis = 1)[i]
        
        
print("----")
print(d)

0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0  
0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.008264462809917356 0.0 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.008729338842975207 0.0 0.0 0.0 0.0  
0.0 0.0 0.009685240185950413 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.010150116219008264 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.01072106025632748 0.0 0.0 0.0 0.0  
0.0 0.0 0.010833094585828543 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.009797274515451478 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.009194214876033058 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.009659090909090909 0.0 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.008729338842975207 0.0 0.0 0.0 0.0  
0.0 0.0 0.0 0.013975192487732437

In [91]:
def get_policy_from_pl(matrix) :
    return np.array(matrix).argmax(axis=1)

policy = get_policy_from_pl(d)

In [92]:
state = env.replace_player_init()
env.render()
done = False
cum_sum = 0
i = 0
max_iter = 30
while not done and i < max_iter: 
    new_state,reward,done = env.step(int(policy[state]))
    cum_sum += reward
    print("------")
    env.render()
    state = new_state
    i += 1
    
print("-------------")
env.render()
print(f"test reward : {cum_sum}")

J 0 0 0 0 0 / 3 / 3 

2 0 0 0 0 0 0 0 / 2 

0 / 2 2 / / 0 2 3 / 

0 2 / / 3 3 / 3 2 0 

0 3 / / 0 0 2 3 0 / 

0 2 0 / / / 2 0 2 0 

0 0 0 / 3 2 0 0 3 2 

2 / 0 2 3 0 0 0 0 2 

3 2 0 0 / 0 0 0 0 0 

3 0 / 0 2 0 2 2 0 0 

------
0 0 0 0 0 0 / 3 / 3 

2 0 J 0 0 0 0 0 / 2 

0 / 2 2 / / 0 2 3 / 

0 2 / / 3 3 / 3 2 0 

0 3 / / 0 0 2 3 0 / 

0 2 0 / / / 2 0 2 0 

0 0 0 / 3 2 0 0 3 2 

2 / 0 2 3 0 0 0 0 2 

3 2 0 0 / 0 0 0 0 0 

3 0 / 0 2 0 2 2 0 0 

------
0 0 0 0 J 0 / 3 / 3 

2 0 0 0 0 0 0 0 / 2 

0 / 2 2 / / 0 2 3 / 

0 2 / / 3 3 / 3 2 0 

0 3 / / 0 0 2 3 0 / 

0 2 0 / / / 2 0 2 0 

0 0 0 / 3 2 0 0 3 2 

2 / 0 2 3 0 0 0 0 2 

3 2 0 0 / 0 0 0 0 0 

3 0 / 0 2 0 2 2 0 0 

------
0 0 0 0 0 0 / 3 / 3 

2 0 0 0 0 0 J 0 / 2 

0 / 2 2 / / 0 2 3 / 

0 2 / / 3 3 / 3 2 0 

0 3 / / 0 0 2 3 0 / 

0 2 0 / / / 2 0 2 0 

0 0 0 / 3 2 0 0 3 2 

2 / 0 2 3 0 0 0 0 2 

3 2 0 0 / 0 0 0 0 0 

3 0 / 0 2 0 2 2 0 0 

------
0 0 0 0 0 0 / 3 / 3 

2 0 0 0 0 0 0 0 / 2 

0 / 2 2 / / 0 2 3 / 

0 2 / / 3 J / 3 2 0 

0 3 